# Valideren van alle karteringen
Maar voor nu enkel groningen

In [1]:
import geopandas as gpd
import pandas as pd
from typing import List, Optional, Dict

from veg2hab import validation

from pathlib import Path
import geopandas as gpd

from veg2hab.waswordtlijst import WasWordtLijst, opschonen_was_wordt_lijst
from veg2hab.definitietabel import DefinitieTabel, opschonen_definitietabel
from veg2hab.vegkartering import Kartering
from veg2hab.fgr import FGR

pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)

/home/jordydelange/.cache/pypoetry/virtualenvs/veg2hab-CuqoUkZb-py3.7/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.11.2-CAPI-1.17.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


### Groningen

In [2]:
validationinfo = pd.read_excel(
    "../data/validationinfo.xlsx", 
    engine='openpyxl', 
    usecols=[
        "naam_in_overzicht_vegkart",
        "naam_in_overzicht_habkart",
        "provincie",
        "path",
        "habtype_cols",
        "percentage_cols",
    ]
)
# Convert all pd.NA to None to prevent having to do "if not pd.isna()" everywhere
validationinfo = validationinfo.applymap(lambda x: None if pd.isna(x) else x)

validationinfo = validationinfo[validationinfo["provincie"] == "Groningen"]
validationinfo["path"] = validationinfo["path"].apply(lambda x: "../testing" + x[1:])
validationinfo

# Load in the karteringen
trues = {}
for row in validationinfo.itertuples():
    raw = gpd.read_file(row.path)
    parsed = validation.parse_habitat_percentages(raw, habtype_cols=row.habtype_cols, percentage_cols=row.percentage_cols, how_to_handle_missing_percentages="split_equally")
    trues[row.naam_in_overzicht_vegkart] = parsed



/home/jordydelange/git/veg2hab/veg2hab/validation.py:76: UserWarning: No non-null habitat types found, returning 100% of H0000
  warnings.warn(f"No non-null habitat types found, returning 100% of H0000")


### Inladen habitatkarteringen

In [3]:
print("Benodigde resources inladen")
path_in_wwl = Path("../data/5. Was-wordt-lijst-vegetatietypen-en-habitattypen-09-02-2021.xlsx")
path_out_wwl = Path("../testing/opgeschoonde_waswordt.xlsx")
opschonen_was_wordt_lijst(path_in_wwl, path_out_wwl)
wwl = WasWordtLijst.from_excel(path_out_wwl)

path_in_dt = Path("../data/definitietabel habitattypen (versie 24 maart 2009)_0.xls")
path_in_mitsjson = Path("../data/mitsjson.csv")
path_out_dt = Path("../testing/opgeschoonde_definitietabel.xlsx")
opschonen_definitietabel(path_in_dt, path_in_mitsjson, path_out_dt)
dt = DefinitieTabel.from_excel(path_out_dt)

fgr = FGR(Path("../data/bronbestanden/FGR.json"))


print("Inladen karteringinfo")
overzicht = pd.read_excel(
    "../data/Overzicht_vegetatiekarteringen.xlsx", 
    sheet_name="Vegetatiekarteringen", 
    engine='openpyxl',
    usecols=[
        "provincie", 
        "naam_kartering", 
        "path_shapes", 
        "landelijke_typologie beschikbaar?", 
        "locatie_landelijke_typologie",
    ],
    dtype="string"
)

# benodigde info voor de access karteringen
access_info = pd.read_excel(
    "../data/accesskarteringinfo.xlsx",
    engine='openpyxl',
    usecols=[
        "naam_kartering",
        "path_csvs",
        "elm_id_name",
    ],
    dtype="string"
)

# benodigde info voor de shapefile karteringen
shape_info = pd.read_excel(
    "../data/shapefilekarteringinfo.xlsx",
    engine='openpyxl',
    usecols=[
        "naam_kartering",
        "ElmID_col",
        "VvN_col",
        "SBB_col",
        "vegtype_split_char",
        "datum_col",
        "opmerking_col",
    ],
    dtype="string"
)
# Convert all pd.NA to None to prevent having to do "if not pd.isna()" everywhere
shape_info = shape_info.applymap(lambda x: None if pd.isna(x) else x)

# Voor nu dus even enkel karteringen uit groningen
overzicht = overzicht[overzicht.provincie == "Groningen"]
overzicht = overzicht[overzicht["landelijke_typologie beschikbaar?"] == "Ja"]
overzicht.path_shapes = overzicht.path_shapes.apply(lambda x: "../testing/vegetatiekarteringen" + x[1:])

access_overzicht = overzicht[overzicht["locatie_landelijke_typologie"] == "access"]
access_overzicht = access_overzicht.join(access_info.set_index("naam_kartering"), on="naam_kartering")
access_overzicht.path_csvs = access_overzicht.path_csvs.apply(lambda x: "../testing/vegetatiekarteringen" + x[1:])

shapefile_overzicht = overzicht[overzicht["locatie_landelijke_typologie"] == "in shapes"]
shapefile_overzicht = shapefile_overzicht.join(shape_info.set_index("naam_kartering"), on="naam_kartering")


print("Inladen karteringen")
karteringen = {}
for row in access_overzicht.itertuples():
    karteringen[row.naam_kartering] = Kartering.from_access_db(Path(row.path_shapes), row.elm_id_name, Path(row.path_csvs))

for row in shapefile_overzicht.itertuples():
    karteringen[row.naam_kartering] = Kartering.from_shapefile(
        Path(row.path_shapes),
        row.ElmID_col,
        VvN_col=row.VvN_col,
        SBB_col=row.SBB_col, 
        vegtype_split_char=row.vegtype_split_char, 
        datum_col=row.datum_col, 
        opmerking_col=row.opmerking_col
    )


print("Toepassen tabellen en checken mitsen")
for kartering in karteringen.values():
    kartering.apply_wwl(wwl)

for kartering in karteringen.values():
    kartering.apply_deftabel(dt)

for kartering in karteringen.values():
    kartering.check_mitsen(fgr)


print("Omzetten naar final format")
preds = {}
for name, kartering in karteringen.items():
    raw = kartering.as_final_format()
    parsed = validation.parse_habitat_percentages(raw)
    preds[name] = parsed

Benodigde resources inladen


/home/jordydelange/.cache/pypoetry/virtualenvs/veg2hab-CuqoUkZb-py3.7/lib/python3.7/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


Alle SBB codes zijn valide
Alle VvN codes zijn valide
Alle SBB codes zijn valide
Alle VvN codes zijn valide
Alle SBB codes zijn valide
Alle VvN codes zijn valide
Inladen karteringinfo
Inladen karteringen
Er zijn 7 vlakken zonder VegTypeInfo. Deze worden verwijderd.
De eerste paar ElmID van de verwijderde vlakken zijn: [6099, 8174, 8174, 11051, 11023]
Er zijn 90 vlakken zonder VegTypeInfo. Deze worden verwijderd.
De eerste paar ElmID van de verwijderde vlakken zijn: [78, 76, 69, 82, 81]
Toepassen tabellen en checken mitsen
Omzetten naar final format


/home/jordydelange/git/veg2hab/veg2hab/validation.py:81: UserWarning: Percentages do not add up to 100% for row: 143, result: defaultdict(<function _convert_row_to_dict.<locals>.<lambda> at 0x7f1161c6b200>, {'HXXXX': 80})
  f"Percentages do not add up to 100% for row: {row.name}, result: {ret_values}"
/home/jordydelange/git/veg2hab/veg2hab/validation.py:81: UserWarning: Percentages do not add up to 100% for row: 154, result: defaultdict(<function _convert_row_to_dict.<locals>.<lambda> at 0x7f1161c6b830>, {'HXXXX': 100.0, 'H0000': 100.0})
  f"Percentages do not add up to 100% for row: {row.name}, result: {ret_values}"
/home/jordydelange/git/veg2hab/veg2hab/validation.py:81: UserWarning: Percentages do not add up to 100% for row: 191, result: defaultdict(<function _convert_row_to_dict.<locals>.<lambda> at 0x7f1161c5fcb0>, {'HXXXX': 100, 'H0000': 100.0})
  f"Percentages do not add up to 100% for row: {row.name}, result: {ret_values}"


### Combineren van de preds en de trues

In [9]:
for key in trues.keys():
    assert key in preds.keys(), f"Key {key} not found in predictions"

output = {}
for key, true in trues.items():
    pred = preds[key]
    combined = validation.spatial_join(pred, true, how="intersection")
    output[key] = validation.voeg_correctheid_toe_aan_df(combined)

/home/jordydelange/git/veg2hab/veg2hab/validation.py:163: UserWarning: Dropping 4193 rows based on area (presumed rounding errors) with a combined area of 2.495387607979196 m²
  f"Dropping {mask.sum()} rows based on area (presumed rounding errors) with a combined area of {overlayed[mask].area.sum()} m²"
/home/jordydelange/git/veg2hab/veg2hab/validation.py:163: UserWarning: Dropping 8208 rows based on area (presumed rounding errors) with a combined area of 0.0 m²
  f"Dropping {mask.sum()} rows based on area (presumed rounding errors) with a combined area of {overlayed[mask].area.sum()} m²"
/home/jordydelange/git/veg2hab/veg2hab/validation.py:163: UserWarning: Dropping 375 rows based on area (presumed rounding errors) with a combined area of 0.3100628544437718 m²
  f"Dropping {mask.sum()} rows based on area (presumed rounding errors) with a combined area of {overlayed[mask].area.sum()} m²"
/home/jordydelange/git/veg2hab/veg2hab/validation.py:163: UserWarning: Dropping 1785 rows based on 

Confusion matrices

In [11]:
for key, gdf in output.items():
    print("------ " + key)
    print(validation.bereken_volledige_conf_matrix(gdf, method="percentage"))
    print("------------------------")

------ SBB Westerwolde 2020
true_hab   H0000  H3130  H3160  H4010A  H4010_A  H4030  H6230  H6410  H6430_A  H7110_B  H7150  H91D0  HXXXX
pred_hab                                                                                                   
H0000     492.38    0.0    0.0     0.0      0.4   2.09   9.16    0.0      0.0      0.0    0.0    0.0    0.0
H3130       0.00    0.0    0.0     0.0      0.0   0.00   0.00    0.0      0.0      0.0    0.0    0.0    0.0
H3160       0.00    0.0    0.0     0.0      0.0   0.00   0.00    0.0      0.0      0.0    0.0    0.0    0.0
H4010A      0.00    0.0    0.0     0.0      0.0   0.00   0.00    0.0      0.0      0.0    0.0    0.0    0.0
H4010_A     0.00    0.0    0.0     0.0      0.0   0.00   0.00    0.0      0.0      0.0    0.0    0.0    0.0
H4030       0.00    0.0    0.0     0.0      0.0   0.00   0.00    0.0      0.0      0.0    0.0    0.0    0.0
H6230       0.00    0.0    0.0     0.0      0.0   0.00   0.00    0.0      0.0      0.0    0.0    0.0    0.0


Saving as gpkg

In [10]:
for key, gdf in output.items():
    gdf.to_file(f"../testing/validation_output/{key}.gpkg")